# Loading data

## Initial Preparation

First define where the training data url is

In [ ]:
import os 
cwd = os.getcwd()
repository_dir = '/'.join(cwd.split('/')[:-1])
TRAIN_DATA_DIR = f'{repository_dir}/data/data/preprocessed/preprocessed.txt'

assert os.path.exists(TRAIN_DATA_DIR), 'Make sure you follow the steps of README.md in data repository'

print(f'Training data located at {TRAIN_DATA_DIR}')

Then load the data

In [ ]:
with open(TRAIN_DATA_DIR, 'r', encoding='utf-8') as train_file:
    train_data = train_file.read()


# Inspection
print(f'Loaded {len(train_data)} characters')
print(f'First 100 characters:\n{train_data[:100]}')

## Tokenization

Tokenization is essentially turning the data into 'words' that the model understands. Below the vocabulary that is defined is based on a character basis. 

TODO: check out other tokenizers:
tiktoken - https://github.com/openai/tiktoken
sentencepiece - https://github.com/google/sentencepiece

In [122]:
chars = sorted(list(set(train_data)))
print(f'Vocabulary ({len(chars)} characters): {"".join(chars)}')
char_to_digit = {char:digit for digit, char in enumerate(chars)}
digit_to_char = {digit:char for digit, char in enumerate(chars)}

encode = lambda string: [char_to_digit[char] for char in string]
decode = lambda digits: ''.join([digit_to_char[digit] for digit in digits])

Vocabulary (115 characters): 
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]_`abcdefghijklmnopqrstuvwxyz{}~ ±Í×çéëñĹ​–—‘’“”•…₈℅→


In [ ]:
import tiktoken

enc = tiktoken.get_encoding('gpt2')